# Tutorial 5:  Creating and detecting a reference

Unless images are taken in a highly standardized environment, e.g. via a scanner or a microscope, variation will be introduced in terms of exposure or distance between camera and photographed object, zooming, etc. To compensate this variation among images within and across datasets, Phenopype contains some preprocessing tools that can automatically correct images.  

## Load project

First we load the project we created in the last tutorial. If we haven't done so, we should create a project: 

In [1]:
import phenopype as pp
import os

proj_dir = r"_temp/my_project"


images = "images"
myproj = pp.project(root_dir=proj_dir) 
myproj.add_files(image_dir=images,
             include="stickle",
             exclude=["side","top"]) 
myproj.add_config(name = "lm", template="landmarks1")
pp.project.save(myproj, overwrite=True)

--------------------------------------------
Phenopype will create a new project at
E:\git_repos\phenopype\tutorials\_temp\my_project

Proceed? (y/n)
y
Warning - project root_dir already exists - overwrite? (y/n)y

"E:\git_repos\phenopype\tutorials\_temp\my_project" created (overwritten)

project attributes written to E:\git_repos\phenopype\tutorials\_temp\my_project\attributes.yaml
--------------------------------------------
--------------------------------------------
phenopype will search for image files at

E:\git_repos\phenopype\tutorials\images

using the following settings:

filetypes: ['jpg', 'JPG', 'jpeg', 'JPEG', 'tif', 'png', 'bmp'], include: stickle, exclude: ['side', 'top'], mode: copy, recursive: False, resize: False, unique: path

Found image stickle1.JPG - phenopype-project folder 0__stickle1 created
Found image stickle2.JPG - phenopype-project folder 0__stickle2 created
Found image stickle3.JPG - phenopype-project folder 0__stickle3 created

Found 3 files
------------

## Create a reference and a template for detection 

With this method you can set a project specific scale by measuring the pixel-to-mm ratio in a reference image. The steps for this are:

1. Click on two points with a known distance (in mm) in between, e.g. on a piece of mm-paper that you put in the image, and hit Enter.
2. Enter the length in mm. This either returns the pixel-to-mm ratio as a float (under low throughput mode), or adds it to all attribute files within a project (under high throughput mode). 
3. Optional: You can mask the reference card with the option `mask=True`, to exclude it from all thresholding algorithms, or use `template=True` to create a template for automatic scale detection that is saved on the project's root directory.

If you used automatic scale detection, the corresponding column in all resulting data frames is "template_px_mm_ratio" for the distance measured in the template, and "current_px_mm_ratio" for the ratio in the current picture detected by "detect_scale".



<center>
<div style="width:500px; text-align: left">
    
![Adding a scale](_assets/images/ex2_scale.gif)
    
</div>
</center>

The reference image can be any image, but choose it carefully: if you plan on doing brightness and colour corrections, it should be in the middle of the distribution of all exposures and colours so corrections will not over-expose or over-saturate the images. 

We will use the image `stickleback_side.jpg` from the `image` folder in `tutorials`:

In [2]:
images = "images"
os.listdir(images)

['cichlid1.jpg',
 'cichlid2.jpg',
 'cichlid3.jpg',
 'cichlid_multi1.jpg',
 'cichlid_multi2.jpg',
 'cichlid_multi3.jpg',
 'isopods.jpg',
 'isopods_fish.mp4',
 'phyto_445.jpg',
 'phyto_469.jpg',
 'phyto_586.jpg',
 'phyto_bright.jpg',
 'snails1.jpg',
 'snails2.jpg',
 'stickle1.JPG',
 'stickle2.JPG',
 'stickle3.JPG',
 'stickleback_side.jpg',
 'stickleback_top.jpg',
 'worms.jpg']

Within a project, the reference image is set with the `add_reference` method of the project object:
1. click on two points inside the provided image
2. enter the distance (returns the pixel-to-mm-ratio)
3. drag a rectangle mask over the reference card 

The pixel-to-mm-ratio for reference image gets saved to every image included in the project, the mask is stored as a template for automatic scale detection with the `find_scale` function. 

In [3]:
myproj.add_reference(reference_image="images/stickleback_side.jpg", 
                     name="scale1",
                     template=True,
                     overwrite=True)

Reference image loaded from images/stickleback_side.jpg
- measure pixel-to-mm-ratio
Reference set
- add column length
Template selected
Reference image saved under E:\git_repos\phenopype\tutorials\_temp\my_project\reference_scale1.tif
Reference image saved under E:\git_repos\phenopype\tutorials\_temp\my_project\reference_template_scale1.tif
Saved reference info to project attributes.
setting active reference to "scale1" for 0__stickle1 (active=True)
setting active reference to "scale1" for 0__stickle2 (active=True)
setting active reference to "scale1" for 0__stickle3 (active=True)


We have now done a few things: 

**First**, we measured pixel-to-mm ratio of our reference information and saved this information to the projects attribute file in the root directory:  

In [4]:
attributes = pp.utils_lowlevel._load_yaml(os.path.join(myproj.root_dir, "attributes.yaml"))
pp.utils_lowlevel._show_yaml(attributes)

project_info:
  date_created: '20210312164805'
  date_changed: '20210312164805'
  phenopype_version: 2.0.dev0
reference:
  scale1:
    date_added: '20210312164819'
    reference_image: reference_scale1.tif
    original_filepath: images/stickleback_side.jpg
    template_px_mm_ratio: 34.306413394582655
    template_image: reference_template_scale1.tif
project_data:
- 0__stickle1
- 0__stickle2
- 0__stickle3


<div class="alert alert-block alert-info">

**NOTE**
    
Don't mind the functions `_load_yaml` and `_show_yaml` loaded from `pp.utils_lowlevel`, as they are only utility functions that work in the background. I use them here to visualize the content of the attributes files.
    
</div>

**Second**, we have created a template of the reference card to be found in all our project's images:

In [5]:
template = pp.load_image(os.path.join(myproj.root_dir, "reference_template_scale1.tif"))
pp.show_image(template)

**Third**, in all our project's image folders, we point towards the reference we created inside the respective `attributes.yaml` files - e.g., for the first image:

In [6]:
attributes = pp.utils_lowlevel._load_yaml(os.path.join(myproj.dirpaths[0], "attributes.yaml"))
pp.utils_lowlevel._show_yaml(attributes)

image_original:
  filename: stickle1.JPG
  width: 2400
  height: 1600
  filepath: E:\git_repos\phenopype\tutorials\images\stickle1.JPG
  filetype: .JPG
image_phenopype:
  date_added: '20210312164805'
  mode: copy
  filename: copy_stickle1.JPG
  width: 2400
  height: 1600
reference:
  project_level:
    scale1:
      active: true


We are now all set to automatically detect the reference card in our images.

## Detecting a reference

To detect the reference template in our images, we need the function `detect_reference` from the `preprocessing module` (for more information, also on the keypoint detection algorithm that powers this function, [refer to the API](api.html#phenopype.core.preprocessing.detect_reference)). For this we just add a new configuration file that contains the `detect_reference` instruction (the "landmarks2" configuration template).

In [7]:
myproj.add_config(name = "lm-scale", 
                  template="landmarks2", 
                  overwrite=True)

New pype configuration created (landmarks2.yaml) from phenopype template:
e:\git_repos\phenopype\phenopype\templates\landmarks2.yaml
pype_lm-scale.yaml created for 0__stickle1
pype_lm-scale.yaml created for 0__stickle2
pype_lm-scale.yaml created for 0__stickle3


Now we run our loop with the new `pype` configuration:  

In [8]:
for directory in myproj.dirpaths:
    p1 = pp.pype(directory, 
           name="lm-scale")

Succesfully loaded existing pype config (pype_config_lm-scale.yaml) from:
E:\git_repos\phenopype\tutorials\_temp\my_project\data\0__stickle1\pype_config_lm-scale.yaml 


------------+++ new pype iteration 2021:03:12 16:48:34 +++--------------


=== AUTOLOAD ===
- project level reference information loaded for scale1
- reference template image loaded from root directory
PREPROCESSING
detect_reference
---------------------------------------------------
Reference card found with 253 keypoint matches:
template image has 34.306413394582655 pixel per mm.
current image has 33.2 pixel per mm.
= 96.834 % of template image.
---------------------------------------------------
MEASUREMENT
landmarks
- setting landmarks
VISUALIZATION
- modifed image
- autoselect canvas
draw_landmarks
EXPORT
save_landmarks
- landmarks saved under E:\git_repos\phenopype\tutorials\_temp\my_project\data\0__stickle1\landmarks_lm-scale.csv.
=== AUTOSAVE ===
save_canvas
- canvas saved under E:\git_repos\phenopype\tutorials

In [10]:
import pandas as pd

lm_df = pd.read_csv(os.path.join(myproj.dirpaths[0], "landmarks_lm-scale.csv"))
lm_df

,filename_original,filename_phenopype,width,height,landmark,x,y
0,stickle1.JPG,copy_stickle1.JPG,2400,1600,1,1250,422
1,stickle1.JPG,copy_stickle1.JPG,2400,1600,2,1389,406
2,stickle1.JPG,copy_stickle1.JPG,2400,1600,3,1720,480


Note the new column `current_px_mm_ratio`, which denotes the pixel-to-mm ratio as per the detected reference template. This information is now also stored in the respective image specific attribute file:

In [11]:
attributes = pp.utils_lowlevel._load_yaml(os.path.join(myproj.dirpaths[0], "attributes.yaml"))
pp.utils_lowlevel._show_yaml(attributes)

image_original:
  filename: stickle1.JPG
  width: 2400
  height: 1600
  filepath: E:\git_repos\phenopype\tutorials\images\stickle1.JPG
  filetype: .JPG
image_phenopype:
  date_added: '20210312164805'
  mode: copy
  filename: copy_stickle1.JPG
  width: 2400
  height: 1600
reference:
  project_level:
    scale1:
      active: true
      detected_px_mm_ratio: 33.2


In [ ]:
Move on to [Tutorial 6](tutorial_5_references.ipynb) to learn how to size and colour adjust all images within a project.